In [1]:
pip install git+https://github.com/yuce/pyswip@master#egg=pyswip

  Cloning https://github.com/yuce/pyswip (to revision master) to /tmp/pip-install-2_74n0xi/pyswip_8c586ef7d57c46448a39c3b177322579
  Running command git clone --filter=blob:none --quiet https://github.com/yuce/pyswip /tmp/pip-install-2_74n0xi/pyswip_8c586ef7d57c46448a39c3b177322579
  Resolved https://github.com/yuce/pyswip to commit 89955611467cec37ab89bf6fbf7f7567cbcb194b
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/home/csgrad/smadani/venvs/general/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
!head kb.txt

Kismet|directed_by|William Dieterle
Kismet|written_by|Edward Knoblock
Kismet|starred_actors|Marlene Dietrich
Kismet|starred_actors|Edward Arnold
Kismet|starred_actors|Ronald Colman
Kismet|starred_actors|James Craig
Kismet|release_year|1944
Kismet|in_language|English
Kismet|has_tags|bd-r
Flags of Our Fathers|directed_by|Clint Eastwood


In [1]:
#TODO: use re
import re

triples = []
rels = set()

with open('kb.txt', 'r') as f:
    kb_data = f.read().strip().split('\n')
    for line in kb_data:
        elems = line.split('|')
        triples.append(elems)
        rels.add(elems[1])
        
print(len(triples))
triples[:5], rels


134741


([['Kismet', 'directed_by', 'William Dieterle'],
  ['Kismet', 'written_by', 'Edward Knoblock'],
  ['Kismet', 'starred_actors', 'Marlene Dietrich'],
  ['Kismet', 'starred_actors', 'Edward Arnold'],
  ['Kismet', 'starred_actors', 'Ronald Colman']],
 {'directed_by',
  'has_genre',
  'has_imdb_rating',
  'has_imdb_votes',
  'has_tags',
  'in_language',
  'release_year',
  'starred_actors',
  'written_by'})

In [2]:
def search_triples(s):
    for triple in triples:
        if s in triple[0] or s in triple[1] or s in triple[2]:
            print(triple)
            
search_triples("john leguizamo")

['Spun', 'has_tags', 'john leguizamo']
['Ice Age', 'has_tags', 'john leguizamo']
['Spawn', 'has_tags', 'john leguizamo']
['Super Mario Bros.', 'has_tags', 'john leguizamo']
['The Pest', 'has_tags', 'john leguizamo']


In [3]:
vocab = {}
for triple in triples:
    for i, ele in enumerate(triple):
        if ele not in vocab:
            if i == 0 or i == 2:
                vocab[ele] = f"ent_{len(vocab)}"
            else:
                vocab[ele] = f"rel_{len(vocab)}"


In [4]:
# add reverse relations

rev_rels = {f"{r}_reverse" for r in rels}
print(rev_rels)

for rel in rev_rels:
    vocab[rel] = f"rel_{len(vocab)}"
    

{'has_imdb_rating_reverse', 'directed_by_reverse', 'has_genre_reverse', 'in_language_reverse', 'has_tags_reverse', 'starred_actors_reverse', 'release_year_reverse', 'written_by_reverse', 'has_imdb_votes_reverse'}


In [6]:
from pyswip import Prolog
from tqdm import tqdm

prolog = Prolog()

for triple in tqdm(triples):
    s1, r, s2 = vocab[triple[0]], vocab[triple[1]], vocab[triple[2]]
    rev_r = vocab[f"{triple[1]}_reverse"]
    prolog.assertz(f"{r}({s1},{s2})")
    prolog.assertz(f"{rev_r}({s2},{s1})")


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134741/134741 [00:04<00:00, 28620.42it/s]


## 1 hop

In [16]:
inv_vocab = {v:k for k, v in vocab.items()}

def query_obj(rel, s1):
    res = list(prolog.query(f"{vocab[rel]}({vocab[s1]},X)"))
    for d in res:
        d['X'] = inv_vocab[d['X']]
    return res

query_obj('written_by', 'Funny About Love')

[{'X': 'Norman Steinberg'},
 {'X': 'David Frankel'},
 {'X': 'Bob Greene'},
 {'X': 'Norman Steinberg'},
 {'X': 'David Frankel'},
 {'X': 'Bob Greene'}]

## 2 hop

In [38]:
f"{vocab['directed_by']}(X,{vocab['Yuriy Norshteyn']}), {vocab['written_by']}(X,Y)"

'rel_1(X,ent_13005), rel_3(X,Y)'

In [37]:
res = list(prolog.query(f"{vocab['directed_by']}(X,{vocab['Yuriy Norshteyn']}), {vocab['written_by']}(X,Y)"))
print(res)

for d in res:
    for k,v in d.items():
        d[k] = inv_vocab[v]
res

[{'X': 'ent_13004', 'Y': 'ent_13005'}, {'X': 'ent_41313', 'Y': 'ent_41314'}]


[{'X': 'Tale of Tales', 'Y': 'Yuriy Norshteyn'},
 {'X': 'Hedgehog in the Fog', 'Y': 'Sergei Kozlov'}]

## dataset to create

How many is enough? to answer I need to read the paper on meta learning

### 1-hop

what films did [Stewart Hopewell] direct -> directed_by(X,[Stewart Hopewell])
who was the director of [Fat Man and Little Boy] -> directed_by([Fat Man and Little Boy],X)
what was the release date of the movie [The Savages] -> release_year([The Savages],X)


### 2-hop


### 3-hop


## weak translation with manual labeling functions

In [46]:
!wc -l ./metaqa/qa_train_3hop.txt
!wc -l ./metaqa/3hop-qtype.txt

114196 ./metaqa/qa_train_3hop.txt
114196 ./metaqa/3hop-qtype.txt


In [13]:
with open("./metaqa/qa_train_3hop.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    questions = []
    for line in lines:
        q, ans = line.split("\t")
        questions.append((q,ans))

with open("./metaqa/3hop-qtype.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    logics = []
    for line in lines:
        logics.append(line)


list(zip(questions, logics))[:5]

[(('the films that share actors with the film [Dil Chahta Hai] were released in which years',
   '1997|1998|2003|2001|2006|2004|2005|2014|2008|2009|2010|2012'),
  'movie_to_actor_to_movie_to_year'),
 (('who are the directors of the movies written by the writer of [The Green Mile]',
   'Stephen King|Frank Darabont|Tobe Hooper|Mick Garris|Lawrence Kasdan|Paul Michael Glaser|John Carpenter|Rob Reiner|Mikael Håfström|Brian De Palma|Chuck Russell|Donald P. Borchers|George A. Romero|Bryan Singer|Tom Holland|Fritz Kiersch|Brett Leonard|Stanley Kubrick|Ralph S. Singleton|Taylor Hackford|Lewis Teague|Mark L. Lester|David Cronenberg|Mark Pavia|David Koepp|Mary Lambert|William Wyler|Michael Gornick|Scott Hicks'),
  'movie_to_writer_to_movie_to_director'),
 (('which person directed the films acted by the actors in [Jawbreaker]',
   "Chris D'Arienzo|Jonathan Kesselman|Joe Chappelle|Robert Rodriguez|Victor Salva|Matthew Leutwyler|Gregg Araki|Mark Duplass|Jared Drake"),
  'movie_to_actor_to_movie_to_

### extract all logical predicates from data

In [8]:

def logics_str_to_steps(logics_str):
    logics_str = logics_str.replace("_to_", "_")
    logics_str += "_"
    steps = []
    logic_steps = []
    i = 0
    prev_und_loc = -1
    while i < len(logics_str):
        if logics_str[i] == '_' or i == len(logics_str)-1:
            steps.append(logics_str[prev_und_loc+1:i])
            prev_und_loc = i
        i+=1
    for i in range(len(steps)-1):
        logic_steps.append(f"{steps[i]}_{steps[i+1]}")
    return logic_steps

logics_str_to_steps('movie_to_actor_to_movie_to_year')

['movie_actor', 'actor_movie', 'movie_year']

In [9]:
logics_strs = []

with open("./metaqa/3hop-qtype.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    for line in lines:
        logics_strs.append(line)
        
with open("./metaqa/2hop-qtype.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    for line in lines:
        logics_strs.append(line)
        
with open("./metaqa/1hop-qtype.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    for line in lines:
        logics_strs.append(line)
        
        
logics_strs_set = set()

for logic_str in logics_strs:
    steps = logics_str_to_steps(logic_str)
    logics_strs_set.update(steps)

logics_strs_set

{'actor_movie',
 'director_movie',
 'movie_actor',
 'movie_director',
 'movie_genre',
 'movie_imdbrating',
 'movie_imdbvotes',
 'movie_language',
 'movie_tags',
 'movie_writer',
 'movie_year',
 'tag_movie',
 'writer_movie'}

### logic step to predicate

In [10]:
logic_to_predicate_dict = {
    'actor_movie': 'starred_actors_reverse',
    'director_movie': 'directed_by_reverse',
    'movie_actor': 'starred_actors',
    'movie_director': 'directed_by',
    'movie_genre': 'has_genre',
    'movie_imdbrating': 'has_imdb_rating',
    'movie_imdbvotes': 'has_imdb_votes',
    'movie_language': 'in_language',
    'movie_tags': 'has_tags',
    'movie_writer': 'written_by',
    'movie_year': 'release_year',
    'tag_movie': 'has_tags_reverse',
    'writer_movie': 'written_by_reverse'
}

In [29]:
import re

def create_query_string(question_str, question_steps, logic_to_predicate_dict, entity_rel_vocab):
    # extract question concept
    question_concept = re.findall(r'\[(.+)\]', question_str)[0]
    #print(question_concept)
    
    # define prolog variables
    prolog_vars = ['X', 'Y', 'Z']
    logic_steps = logics_str_to_steps(question_steps)
    predicate_steps = [logic_to_predicate_dict[x] for x in logic_steps]
    #print(logic_steps)
    #print(predicate_steps)
    
    # create predicates
    p1 = f'{predicate_steps[0]}({question_concept},X)'
    p1_query = f'{entity_rel_vocab[predicate_steps[0]]}({entity_rel_vocab[question_concept]},X)'
    i = 1
    predicates = [p1,]
    predicates_query = [p1_query,]
    while i < len(predicate_steps):
        predicates.append(f"{predicate_steps[i]}({prolog_vars[i-1]},{prolog_vars[i]})")
        predicates_query.append(
            f'{entity_rel_vocab[predicate_steps[i]]}({prolog_vars[i-1]},{prolog_vars[i]})'
        )
        i += 1
        
    query_string = ', '.join(predicates_query)
    return predicates, query_string
    
predicates, query = create_query_string(
    'what types are the films directed by the director of [For Love or Money]',
    'movie_to_director_to_movie_to_genre',
    logic_to_predicate_dict,
    vocab
)

In [18]:
def interpret_query_result(query_result, eid_to_entity_dict):
    interpreted_res = []
    for result in query_result:
        d = {}
        for key, val in result.items():
            d[key] = eid_to_entity_dict[val]
        interpreted_res.append(d)
    return interpreted_res

print(predicates, query)
res=list(prolog.query(query))
interpret_query_result(res, inv_vocab)

['directed_by(For Love or Money,X)', 'directed_by_reverse(X,Y)', 'has_genre(Y,Z)'] rel_1(ent_23998,X), rel_43244(X,Y), rel_22(Y,Z)


[{'X': 'Barry Sonnenfeld', 'Y': 'Addams Family Values', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Addams Family Values', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Big Trouble', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Big Trouble', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Big Trouble', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Big Trouble', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'For Love or Money', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'For Love or Money', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'RV', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'RV', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Men in Black 3', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Men in Black 3', 'Z': 'Action'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Men in Black 3', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Men in Black 3', 'Z': 'Action'},
 {'X': 'Barry Sonnenfeld', 'Y': 'Get Shorty', 'Z': 'Comedy'},
 {'X': 'Barry Sonnenfeld', 'Y': 

### build the dataset

In [23]:
questions = []
logics = []

for i in range(1,4):
    with open(f"./metaqa/qa_train_{i}hop.txt", 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines:
            q, ans = line.split("\t")
            questions.append((q,ans))

    with open(f"./metaqa/{i}hop-qtype.txt", 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines:
            logics.append(line)
        


list(zip(questions, logics))[:5]

[(('what movies are about [ginger rogers]',
   'Top Hat|Kitty Foyle|The Barkleys of Broadway'),
  'tag_to_movie'),
 (('which movies can be described by [moore]',
   'Fahrenheit 9/11|Far from Heaven'),
  'tag_to_movie'),
 (('what films can be described by [occupation]',
   'Red Dawn|The Teahouse of the August Moon'),
  'tag_to_movie'),
 (('which films are about [jacques tati]', 'Mon Oncle|Playtime|Trafic'),
  'tag_to_movie'),
 (('what movies are about [donnie darko]', 'S. Darko'), 'tag_to_movie')]

In [25]:
from sklearn.model_selection import train_test_split

raw_dataset = list(zip(questions, logics))
train_ds, test_ds = train_test_split(raw_dataset, test_size=0.05)
len(train_ds), len(test_ds)

(312817, 16465)

In [26]:
train_ds[:5]

[(('describe the movie [Bad Girls] in a few words',
   'drew barrymore|andie macdowell|mary stuart masterson'),
  'movie_to_tags'),
 (('what genres do the movies written by [Harriet Frank Jr.] fall under',
   'Romance|Comedy'),
  'writer_to_movie_to_genre'),
 (('the films directed by the director of [Heaven Is for Real] starred who',
   'Joseph Schildkraut|Diane Lane|Joan Bennett|Louis Hayward|Jeremy Irons|Warren William|Leonardo DiCaprio|Mel Gibson|John Malkovich'),
  'movie_to_director_to_movie_to_actor'),
 (('the movies starred by [Diedrich Bader] were written by who',
   'Aaron Seltzer|Jason Friedberg'),
  'actor_to_movie_to_writer'),
 (('the actor [Heather Donahue] co-starred with who',
   'Michael C. Williams|Joshua Leonard'),
  'actor_to_movie_to_actor')]

In [34]:
import pandas as pd

def build_csv_dataset(ds, path):
    translated = []
    questions = []
    for (question, ans), logic in ds:
        questions.append(question)
        predicates, query = create_query_string(
            question,
            logic,
            logic_to_predicate_dict,
            vocab
        )
        translated.append(",".join(predicates))
    df = pd.DataFrame({'question': questions, 'logic': translated})
    #df.to_csv(path, index=False, header=True)
    df.to_json(path, orient='records', lines=True)

build_csv_dataset(train_ds, 'train_s2s.jsonl')
build_csv_dataset(test_ds, 'valid_s2s.jsonl')

In [1]:
!head train_s2s.json

{"question":"describe the movie [Bad Girls] in a few words","logic":"has_tags(Bad Girls,X)"}
{"question":"what genres do the movies written by [Harriet Frank Jr.] fall under","logic":"written_by_reverse(Harriet Frank Jr.,X),has_genre(X,Y)"}
{"question":"the films directed by the director of [Heaven Is for Real] starred who","logic":"directed_by(Heaven Is for Real,X),directed_by_reverse(X,Y),starred_actors(Y,Z)"}
{"question":"the movies starred by [Diedrich Bader] were written by who","logic":"starred_actors_reverse(Diedrich Bader,X),written_by(X,Y)"}
{"question":"the actor [Heather Donahue] co-starred with who","logic":"starred_actors_reverse(Heather Donahue,X),starred_actors(X,Y)"}
{"question":"what types are the movies written by [St\u00e9phane Aubier]","logic":"written_by_reverse(St\u00e9phane Aubier,X),has_genre(X,Y)"}
{"question":"who are the actors in the films directed by the director of [Les Mistons]","logic":"directed_by(Les Mistons,X),directed_by_reverse(X,Y),starred_actors(Y

## test model

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./results/t5-v1/checkpoint-4000/")

In [13]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("./results/t5-v1/checkpoint-4000/")

In [17]:
sent = "summarize: the films directed by the director of [Heaven Is for Real] starred who"
#,"logic":"directed_by(Heaven Is for Real,X),directed_by_reverse(X,Y),starred_actors(Y,Z)""
inputs = tokenizer(
    f"summarize: {sent}",
    return_tensors="pt",
#     max_length=100,
#     truncation=True
).input_ids
outputs = model.generate(inputs, max_new_tokens=200)#, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

directed_by(Heaven Is for Real,X), directed_by_reverse(X,Y), starred_actors(Y,Z)


In [18]:
model.config

T5Config {
  "_name_or_path": "./results/t5-v1/checkpoint-4000/",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num

In [19]:
inputs = tokenizer(
    [f"summarize: {sent}", "which mother fuckers played in movie [navid in wonderland]"],
    return_tensors="pt",
    max_length=100,
    truncation=True,
    padding=True
).input_ids

outputs = model.generate(inputs, max_length=50)

In [20]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(tokenizer.decode(outputs[1], skip_special_tokens=True))

directed_by(Heaven Is for Real,X), directed_by_reverse(X,Y), starred_actors(Y,Z)
starred_actors(navid in wonderland,X)


### accuracy of translator on test set

In [21]:
import json

questions = []
trans_qs = []
with open('data/test.json', 'r') as json_file:
    json_list = list(json_file)
    for json_str in json_list:
        result = json.loads(json_str)
        questions.append(result['question'])
        trans_qs.append(result['logical_steps'])

# preprocess questions
questions = [f"summarize: {q}" for q in questions]
questions[:3], len(questions)

(['summarize: what does [Grégoire Colin] appear in',
  'summarize: [Joe Thomas] appears in which movies',
  'summarize: what films did [Michelle Trachtenberg] star in'],
 39093)

In [25]:
from tqdm import tqdm

def get_translated_output(query_list, tokenizer, model, batch_size=5000):
    results = []
    i = 0
    pbar = tqdm(total = len(query_list))
    
    while i < len(query_list):
        batch = query_list[i:i+batch_size]
        
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            max_length=200,
            truncation=True,
            padding=True
        ).input_ids
        
        outputs = model.generate(inputs, max_length=200)
        
        curr_res = [tokenizer.decode(out, skip_special_tokens=True) for out in outputs]
        i += batch_size
        pbar.update(batch_size)
        results.extend(curr_res)
    pbar.close()
    return results
    
model_outputs = get_translated_output(questions, tokenizer, model)


 90%|████████▉ | 35000/39093 [03:53<00:29, 138.31it/s]
40000it [04:16, 156.18it/s]                           


In [26]:
exact_matches = 0

for model_gen, gold_out in zip(model_outputs, trans_qs):
    if model_gen == gold_out:
        exact_matches += 1
    else:
        print(model_gen, gold_out)
        
print(f"accuracy: {exact_matches/len(model_outputs)}")

starred_actors_reverse(Martn Adjemián,X) starred_actors_reverse(Martín Adjemián,X)
starred_actors_reverse(Frank Ramrez,X) starred_actors_reverse(Frank Ramírez,X)
starred_actors_reverse(Alicia Rodrguez,X) starred_actors_reverse(Alicia Rodríguez,X)
starred_actors_reverse(Gwenalle Simon,X) starred_actors_reverse(Gwenaëlle Simon,X)
starred_actors_reverse(Manuel Manquia,X) starred_actors_reverse(Manuel Manquiña,X)
starred_actors_reverse(Ma Maestro,X) starred_actors_reverse(Mía Maestro,X)
starred_actors_reverse(Fabrcio Boliveira,X) starred_actors_reverse(Fabrício Boliveira,X)
directed_by_reverse(Peter Schnau Fog,X) directed_by_reverse(Peter Schønau Fog,X)
directed_by_reverse(Jol Séria,X) directed_by_reverse(Joël Séria,X)
directed_by_reverse(Icar Bollan,X) directed_by_reverse(Icíar Bollaín,X)
directed_by_reverse(Rigoberto Castaeda,X) directed_by_reverse(Rigoberto Castañeda,X)
directed_by_reverse(Jonas kerlund,X) directed_by_reverse(Jonas Åkerlund,X)
directed_by_reverse(Hafsteinn Gunnar Sigurs

In [32]:
import pickle

with open('predictions.pkl', 'wb') as f:
    pickle.dump(model_outputs, f)

In [27]:
len(model_outputs), len(trans_qs)

(39093, 39093)